In [195]:
#import dependencies
import numpy
import sys
import nltk
nltk.download('stopwords')
import tensorflow
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [196]:
#load data
#loading data and opening out input data inthe form of a text file
#Project Gutenburg/berg is where the data can be found(Just Google it:)
file = open ("frankenstein-2.txt").read()

In [248]:
#tokenization
#standardization
#what is tokenization ? Tokenization is a process of breaking a stream of text up into words phrases symbols or a meaningful elements
def tokenize_words(input):
    #lowercase everything  to standardize it
    input = input.lower()
    #instanting the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    #tokenizing the text into tokens
    tokens = tokenizer.tokenize(input)
    #filtering the stopwords using  lambda
    filtered = filter(lambda token: token not in stopwords.words('english'),tokens)
    return " ".join(filtered)

#preprocess the input data, make tokens
processed_inputs = tokenize_words(file)    

In [249]:
#chars to numbers
#convert characters in our input to numbers
#we'll sort the list of the set of all characters that appear in out i/p text and then use the enumerate fn to get numbers to that
#represent the characters
# we'll then create a dictionary that stores the keys and values, or the characters and the numbers that represent them
chars =  sorted(list(set(processed_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [250]:
#check if words to chars or chars to num (71) has worked?
#just so we get an idea of whether our process of converting words to characters has worked, we print the length of our variables
input_len = len(processed_inputs)
vocab_len = len(chars)
print("Total number of characters:", input_len)
print("Total vocab:", vocab_len)

Total number of characters: 269995
Total vocab: 43


In [251]:
#seg length
#we're defining how long we want an indivisual sequence here
#in indivisual sequence is a complete mapping of input characters as integers
seq_length =100
x_data =[]
y_data =[]

In [252]:
#loop through the sequence
#here we're going through the entire list of i/ps and converting the chars  to numbers with a for loop
#this will create a bunch of sequence where each sequence starts with next characters in the i/p data beginning with the first char
for i in range(0, input_len - seq_length,1):
    #define i/p and o/p sequences
    #i/p is the current char plus the desired sequence length
    in_seq = processed_inputs[i:i+seq_length]
    #out sequences is the initial characters plus total sequence length
    out_seq = processed_inputs[i+seq_length]
    #converting the list of characters to integers based on previous values and appending the values to our lists 
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
#check to see how many total input sequences we have   
n_patterns = len(x_data)
print("Total Patterns:", n_patterns)

Total Patterns: 269895


In [253]:
#convert input sequence to np array that our network can use
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [254]:
# one-hot encoding our label data
y= np_utils.to_categorical(y_data)

In [255]:
#creating the model
#creating a sequential model
#dropout is used to prevent overfitting
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences= True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences= True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [256]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [257]:
# saving weights
filepath="model_weights_saved.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks=[checkpoint]

In [258]:
#fit model and let it train
model.fit(X,y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
1055/1055 [==============================] - ETA: 0s - loss: 2.9139
Epoch 00001: loss improved from inf to 2.91389, saving model to model_weights_saved.hdf5
1055/1055 [==============================] - 168s 160ms/step - loss: 2.9139
Epoch 2/4
1055/1055 [==============================] - ETA: 0s - loss: 2.6247
Epoch 00002: loss improved from 2.91389 to 2.62468, saving model to model_weights_saved.hdf5
1055/1055 [==============================] - 168s 159ms/step - loss: 2.6247
Epoch 3/4
1055/1055 [==============================] - ETA: 0s - loss: 2.4752
Epoch 00003: loss improved from 2.62468 to 2.47524, saving model to model_weights_saved.hdf5
1055/1055 [==============================] - 168s 160ms/step - loss: 2.4752
Epoch 4/4
1055/1055 [==============================] - ETA: 0s - loss: 2.3539
Epoch 00004: loss improved from 2.47524 to 2.35393, saving model to model_weights_saved.hdf5
1055/1055 [==============================] - 168s 159ms/step - loss: 2.3539


In [259]:
#recompile model with the saved weights
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [260]:
#output of the model back into characters
num_to_char = dict((i,c) for i,c in enumerate(chars))

In [261]:
#random seed to help gererate 
start= numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed: ")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed: 
" eserts scotland endured incalculable fatigue cold hunger dare destroy hopes begone break promise nev "


In [262]:
#generate the text
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x= x/float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern= pattern[1:len(pattern)]

er seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare seare s